## How to run your customized workflow?

### Requirement
You need to have an OpenAI API Key.

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "Your API Key"

### Set overall task & Customize your own workflow
Firstly, you need to define your overall task. After that, you can freely define your own workflow.  
Following attributes need to be provided for each subtask:
- **`id`**: The identifier for this subtask.  
  Format: `"taskx"`, where `x` is an Arabic numeral (e.g., `task1`, `task2`).

- **`objective`**: The specific content or goal of this subtask.

- **`agent_id`**: The identifier for the agent responsible for executing this subtask.

- **`next`**: A list of tasks that depend on the completion of this subtask.  
  Values in this list must be `id`s of previously defined tasks.

- **`prev`**: A list of tasks that this subtask depends on.  
  Values in this list must be `id`s of previously defined tasks.

- **`agent`**: The name of the agent responsible for completing this subtask.  
  The same `agent_id` must always correspond to the same `agent`.

Note That：
- Your workflow **must not** contain any form of circular dependencies.
- Ensure that all `next` and `prev` values reference valid `id`s of previously defined tasks.
- The `agent_id` and `agent` must remain consistent across all tasks.

In [1]:
from workflow import Task, Workflow

# Customize your overall task here.
overall_task: str = '''Develop a Rock-Paper-Scissors game with a graphical user interface (GUI) in Python. The game should allow a player to compete against a naive AI that randomly selects Rock, Paper, or Scissors. The UI should display the player’s choice, the AI’s choice, and the game result (win, lose, or draw). Provide an interactive and user-friendly experience.'''

# Use a list of dictionaries to initialize task data here.
tasks_data = [
    {
        "id": "task0",
        "objective": "Design and implement the game's user interface using tkinter, including buttons, computer's choice and result display in python.",
        "agent_id": 0,
        "next": ["task2"],
        "prev": [],
        "agent": "UI Designer",
    },
    {
        "id": "task1",
        "objective": "Design a naive AI and core game logic to determine the winner based on player and AI choices in python.",
        "agent_id": 1,
        "next": ["task2"],
        "prev": [],
        "agent": "Game Logic Developer",
    },
    {
        "id": "task2",
        "objective": "Combine the UI and game logic, test the game, and prepare it for deployment.",
        "agent_id": 2,
        "next": [],
        "prev": ["task0", "task1"],
        "agent": "Integration Specialist",
    },
]

# Create the workflow
workflow = Workflow(tasks={task_data["id"]: Task(**task_data) for task_data in tasks_data})

### Run your workflow

In [3]:
import asyncio
import nest_asyncio
import json
from flow import Flow
from summary import Summary



manager = Flow(overall_task=overall_task, enable_refine=False, refine_threhold=0, n_candidate_graphs=0, workflow=workflow)
nest_asyncio.apply()
asyncio.run(manager.run_async())


workflow_data = {
    tid: task.__dict__ for tid, task in manager.workflow.tasks.items()
}
with open('manually_result.json', 'w', encoding='utf-8') as file:
    json.dump(workflow_data, file, indent=4)

summary = Summary()

# Generate and save a summary of the workflow results
chat_result = summary.summary(overall_task, workflow_data)
with open("example.txt", "w", encoding="utf-8") as file:
    file.write(chat_result)

2025-02-19 19:21:09 [INFO] Executing task 'Design and implement the game's user interface using tkinter, including buttons, computer's choice and result display in python.' with context: No completed previous tasks context available....
2025-02-19 19:21:09 [INFO] Task 'Design and implement the game's user interface using tkinter, including buttons, computer's choice and result display in python.' started by agent '0'.
2025-02-19 19:21:09 [INFO] Executing task 'Design a naive AI and core game logic to determine the winner based on player and AI choices in python.' with context: No completed previous tasks context available....
2025-02-19 19:21:09 [INFO] Task 'Design a naive AI and core game logic to determine the winner based on player and AI choices in python.' started by agent '1'.


object: Design and implement the game's user interface using tkinter, including buttons, computer's choice and result display in python.
next: Task task2:
  Objective: Combine the UI and game logic, test the game, and prepare it for deployment.
object: Design a naive AI and core game logic to determine the winner based on player and AI choices in python.
next: Task task2:
  Objective: Combine the UI and game logic, test the game, and prepare it for deployment.


2025-02-19 19:21:14 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-19 19:21:14 [INFO] Task task1 completed with result: {
  "naive_ai": {
    "description": "A simple AI that randomly selects Rock, Paper, or Scissors.",
    "implementation": {
      "import_statement": "import random",
      "function": {
        "name": "get_ai_choice",
        "return_type": "str",
        "return_values": [
          "Rock",
          "Paper",
          "Scissors"
        ],
        "code": "def get_ai_choice():\n    return random.choice(['Rock', 'Paper', 'Scissors'])"
      }
    }
  },
  "game_logic": {
    "description": "Core game logic to determine the winner based on player and AI choices.",
    "function": {
      "name": "determine_winner",
      "parameters": [
        {
          "name": "player_choice",
          "type": "str"
        },
        {
          "name": "ai_choice",
          "type": "str"
        }
      ],
      "return_type": 

1 inf


2025-02-19 19:21:21 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-19 19:21:21 [INFO] Task task0 completed with result: {
  "subtask": {
    "description": "Design and implement the game's user interface using tkinter for Rock-Paper-Scissors.",
    "implementation": {
      "language": "Python",
      "libraries": ["tkinter"],
      "code": "import tkinter as tk\nimport random\n\nclass RockPaperScissors:\n    def __init__(self, master):\n        self.master = master\n        master.title('Rock-Paper-Scissors')\n\n        self.player_choice = None\n        self.ai_choice = None\n        self.result = None\n\n        self.label = tk.Label(master, text='Choose Rock, Paper, or Scissors:', font=('Arial', 14))\n        self.label.pack()\n\n        self.rock_button = tk.Button(master, text='Rock', command=lambda: self.play('Rock'), font=('Arial', 12))\n        self.rock_button.pack(pady=5)\n\n        self.paper_button = tk.Button(master, text='P

2 inf
object: Combine the UI and game logic, test the game, and prepare it for deployment.
next: No downstream objectives available.


2025-02-19 19:21:36 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-19 19:21:36 [INFO] Task task2 completed with result: {
  "game_combination": {
    "description": "Integrate the user interface with the game logic to create a fully functional Rock-Paper-Scissors game.",
    "implementation": {
      "language": "Python",
      "libraries": ["tkinter", "random"],
      "code": [
        "import tkinter as tk",
        "import random",
        "",
        "def get_ai_choice():",
        "    return random.choice(['Rock', 'Paper', 'Scissors'])",
        "",
        "def determine_winner(player_choice, ai_choice):",
        "    if player_choice == ai_choice:",
        "        return 'Draw'",
        "    elif (player_choice == 'Rock' and ai_choice == 'Scissors') or",
        "         (player_choice == 'Paper' and ai_choice == 'Rock') or",
        "         (player_choice == 'Scissors' and ai_choice == 'Paper'):",
        "        return '

3 inf


2025-02-19 19:21:46 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [4]:
import tkinter as tk
import random

def get_ai_choice():
    """A simple AI that randomly selects Rock, Paper, or Scissors."""
    return random.choice(['Rock', 'Paper', 'Scissors'])

def determine_winner(player_choice, ai_choice):
    """Core game logic to determine the winner based on player and AI choices."""
    if player_choice == ai_choice:
        return 'Draw'
    elif (player_choice == 'Rock' and ai_choice == 'Scissors') or \
         (player_choice == 'Paper' and ai_choice == 'Rock') or \
         (player_choice == 'Scissors' and ai_choice == 'Paper'):
        return 'You Win'
    else:
        return 'You Lose'

class RockPaperScissors:
    def __init__(self, master):
        self.master = master
        master.title('Rock-Paper-Scissors')

        self.player_choice = None
        self.ai_choice = None
        self.result = None

        self.label = tk.Label(master, text='Choose Rock, Paper, or Scissors:', font=('Arial', 14))
        self.label.pack()

        self.rock_button = tk.Button(master, text='Rock', command=lambda: self.play('Rock'), font=('Arial', 12))
        self.rock_button.pack(pady=5)

        self.paper_button = tk.Button(master, text='Paper', command=lambda: self.play('Paper'), font=('Arial', 12))
        self.paper_button.pack(pady=5)

        self.scissors_button = tk.Button(master, text='Scissors', command=lambda: self.play('Scissors'), font=('Arial', 12))
        self.scissors_button.pack(pady=5)

        self.result_label = tk.Label(master, text='', font=('Arial', 14))
        self.result_label.pack(pady=20)

    def play(self, player_choice):
        self.player_choice = player_choice
        self.ai_choice = get_ai_choice()
        self.result = determine_winner(self.player_choice, self.ai_choice)
        self.display_result()

    def display_result(self):
        self.result_label.config(text=f'You chose: {self.player_choice}\nAI chose: {self.ai_choice}\nResult: {self.result}')

if __name__ == '__main__':
    root = tk.Tk()
    game = RockPaperScissors(root)
    root.mainloop()